# CRM Query Engine Test
This notebook demonstrates the implementation and usage of the SQL query engine.

In [1]:
from currensee.schema.schema import PostgresTables
from currensee.query_engines.sql_query_engine.query_engine import create_sql_workflow
from currensee.utils.db_utils import create_pg_engine
from currensee.query_engines.workflow_descriptions import crm_portfolio_table_desc, crm_client_alignment_table_desc,crm_client_info_table_desc, crm_employees_table_desc,crm_fund_details_desc 

In [2]:
# required to run asynchronous code

import nest_asyncio

nest_asyncio.apply()

## Create the SQL Workflow

The SQL workflow can take the following parameters:

1. source_db: the name of the database where the table is stored (e.g. `crm`)
2. source_tables: a list of the name(s) of the table(s) that we want the query engine to have access to
  * note that multiple tables can be passed - this is if you want the query engine to try to join tables
    in the queries that may have relationships to one another
  * THIS IS LEVEL 2!! So do not attempt until you get the hang of just using one table at a time!!
    
3. table_descriptions: a list of the description(s) of the table(s) passed above
4. text_to_sql_tmpl: a string containing the prompt telling the LLM how to produce the SQL query from the text given
   * defaults to the variable `text_to_sql_tmpl` defined in `currensee.query_engines.prompting.py`
   * you may override this by passing in your own string
5. response_synthesis_prompt_str: a string containing the prompt telling the LLM how to synthesize the final response from the SQL table(s)
   * defaults to the variable `response_synthesis_prompt_str` defined in `currensee.query_engines.prompting.py`
   * you may override this by passing in your own string
6. model: the name of the model to use for all of the tasks
   * defaults to `gemini-1.5-flash`
   * you may override this with any of the models defined at https://ai.google.dev/gemini-api/docs/models#model-variations using the string with dashes defined in the "Model variant" column.
   * **BE VERY CAREFUL TO PAY ATTENTION TO THE PRICING!!!!!** I recommend that you use the default model until you understand the other models better!!!
7. temperature: the temperature parameter to pass to the model
   * default is 0.0
   * the higher the temperature, the more creative it is. Recommend keeping low for the SQL query generation.

### Below is the default defined in `prompting.py`

In [3]:
text_to_sql_tmpl = """
    Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
    You can order the results by the find_date column (from earliest to latest) to return the most interesting examples in the database.

    GUIDELINES:
    * Never query for all the columns from a specific table, only ask for a few relevant columns given the question.
    * Pay attention to use only the column names that you can see in the schema description.
    * Be careful to not query for columns that do not exist.
    * Pay attention to which column is in which table.
    * Make sure to filter on all criteria mentioned in the query.
    * If using a LIMIT to restrict the results, make sure it comes only in the end of the query.

    IMPORTANT NOTE:
    * Use the ~* operator instead of = when filtering with WHERE on text columns.
    * Add word boundaries '\y' to the beginning and end of each search term in the query.

    You are required to use the following format, each taking one line:

    Question: Question here
    SQLQuery: SQL Query to run
    SQLResult: Result of the SQLQuery
    Answer: Final answer here

    Only use tables listed below.
    {schema}

    Question: {query_str}
    SQLQuery:

"""

### Below is the default defined in `prompting.py`

In [4]:
response_synthesis_prompt_str = """

    Query: {query_str}
    SQL: {sql_query}
    SQL Response: {context_str}

    IMPORTANT INSTRUCTIONS:
    * If SQL Response is empty or 0, apologise and mention that you could not find
     examples to answer the query.
    * In such cases, kindly nudge the user towards providing more details or refining
    their search.
    * Additionally, you can tell them to rephrase specific keywords.
    * Do not explicitly state phrases such as 'based on the SQL query executed' or related
     references to context in your Response.
    * Never mention the underlying sql query, or the underling sql tables and other database elements
    * Never mention that sql was used to answer this question

    Considering the IMPORTANT INSTRUCTIONS above, create an response using the information
    returned from the database and no prior knowledge.


    Response:
"""

### Define the DB information
**IMPORTANT**: The table names MUST be lowercase in order for the engine to find them.

In [5]:
source_db = 'crm'
table_description_mapping = {
    'employees': crm_employees_table_desc,
    'portfolio': crm_portfolio_table_desc,
    'fund_detail': crm_fund_details_desc,
    'client_alignment': crm_client_alignment_table_desc,
    'clients_contact': crm_client_info_table_desc
}

In [6]:
sql_workflow = create_sql_workflow(
    source_db = source_db,
    table_description_mapping=table_description_mapping,
    text_to_sql_tmpl=text_to_sql_tmpl,
    response_synthesis_prompt_str=response_synthesis_prompt_str
    
)

# Test Queries

In [7]:
query = "Who works for bankwell?"
result = await sql_workflow.run(query=query)
result

Running step generate_sql_response
Step generate_sql_response produced event StopEvent


Response(response='The following individuals work at Bankwell: Jane Moneypenny, Russell Sherman, Veronica West, Marie Howard, Debra Kelly, Meghan Wiggins, Christopher Flores, Brandon Hernandez, Kevin Rocha, Karen Smith, Jennifer Ayala, Mark Davis, Kenneth Padilla, Mary Martinez, Jay Baker, Thomas Walter, Timothy Dyer, Teresa Carroll, Jacob Jennings, Daniel Wallace, Tonya Kidd, David Jones, Pamela Spencer, Amanda Allen, Adam Rodriguez, Thomas Watkins, Sabrina Gregory, Daniel Sawyer, Kathleen Kelley, Alexandria Collins, James Wolfe, Janice Tucker, Laura Brown, Lisa Mccormick, Michelle Mcbride, Nicholas Garcia, Garrett Swanson, Michael Wilson, Jessica Hanson, Michelle Smith, Jennifer Miller, Julie Jimenez, Alicia Peterson, Rhonda Jenkins, Peter Marshall, Laurie Turner, Benjamin Gilbert, Stephanie Gutierrez, Kelsey Charles, Jesus Yates, Matthew Stephenson, Samantha Barnes, Ann Mcdaniel, Catherine Hunter, Stephanie Garner, Anthony Mitchell, Julie Harmon, Amanda Gonzales, Karen Flores, Melin

In [8]:
query = "How many people work for bankwell in the Sales department?"
result = await sql_workflow.run(query=query)
result

Running step generate_sql_response
Step generate_sql_response produced event StopEvent


Response(response='There are 29 people working in the Sales department at Bankwell.\n', source_nodes=[NodeWithScore(node=TextNode(id_='33760e33-a092-47d6-9a16-ddf1bea83dd9', embedding=None, metadata={'sql_query': "SELECT count(*) FROM employees WHERE department ~* '\\ySales\\y';", 'result': [(29,)], 'col_keys': ['count']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='[(29,)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)], metadata={'33760e33-a092-47d6-9a16-ddf1bea83dd9': {'sql_query': "SELECT count(*) FROM employees WHERE department ~* '\\ySales\\y';", 'result': [(29,)], 'col_keys': ['count']}, 'sql_query': "SELECT count(*) FROM employees WHERE department ~* '\\ySales\\y';", 'result': [(29,)], 'col_keys': ['count'], 'u

In [9]:
query = "what financial instruments does Mariott own?"
result = await sql_workflow.run(query=query)
result

Running step generate_sql_response
Step generate_sql_response produced event StopEvent


Response(response='Marriott appears to hold investments in Bond Funds (specifically, BND and TLT) and an Equity Fund (VSMPX).\n', source_nodes=[NodeWithScore(node=TextNode(id_='e641ce32-6c08-4bcf-8a7e-f13ddfab4af6', embedding=None, metadata={'sql_query': "SELECT symbol, fund_type FROM portfolio WHERE company ~* '\\yMariott\\y'", 'result': [('BND', 'Bond Fund'), ('VSMPX', 'Equity Fund'), ('TLT', 'Bond Fund')], 'col_keys': ['symbol', 'fund_type']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="[('BND', 'Bond Fund'), ('VSMPX', 'Equity Fund'), ('TLT', 'Bond Fund')]", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)], metadata={'e641ce32-6c08-4bcf-8a7e-f13ddfab4af6': {'sql_query': "SELECT symbol, fund_type FROM portfolio WHERE co

In [10]:
query = "how many funds does Broadcom own? What types of funds are they?"
result = await sql_workflow.run(query=query)
result

Running step generate_sql_response
Step generate_sql_response produced event StopEvent


Response(response='Broadcom appears to own 5 distinct funds, encompassing 2 different fund types.\n', source_nodes=[NodeWithScore(node=TextNode(id_='83eb07cd-a69a-48e9-b929-f4b9c877a114', embedding=None, metadata={'sql_query': "SELECT COUNT(DISTINCT symbol), COUNT(DISTINCT fund_type) FROM portfolio WHERE company ~* '\\yBroadcom\\y';", 'result': [(5, 2)], 'col_keys': ['count', 'count']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='[(5, 2)]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)], metadata={'83eb07cd-a69a-48e9-b929-f4b9c877a114': {'sql_query': "SELECT COUNT(DISTINCT symbol), COUNT(DISTINCT fund_type) FROM portfolio WHERE company ~* '\\yBroadcom\\y';", 'result': [(5, 2)], 'col_keys': ['count', 'count']}, 'sql_quer

In [11]:
query = "What is Broadcom's total balance? What fund does broadcom have the most money in, and how much is there?"
result = await sql_workflow.run(query=query)
result

Running step generate_sql_response
Step generate_sql_response produced event StopEvent


Response(response="Broadcom's total balance is $38,560,125.  The fund with the largest amount of Broadcom's money is VSMPX, holding $18,000,155.45.\n", source_nodes=[NodeWithScore(node=TextNode(id_='3385c962-8bf3-4443-9ce4-36efb6be0607', embedding=None, metadata={'sql_query': "SELECT tot_balance, symbol, fund_balance FROM portfolio WHERE company ~* '\\yBroadcom\\y' ORDER BY fund_balance DESC LIMIT 1;", 'result': [(38560125, 'VSMPX', 18000155.45)], 'col_keys': ['tot_balance', 'symbol', 'fund_balance']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="[(38560125, 'VSMPX', 18000155.45)]", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)], metadata={'3385c962-8bf3-4443-9ce4-36efb6be0607': {'sql_query': "SELECT tot_balance, symbol,

In [12]:
query = "How much money does broadcom have in each fund?"
result = await sql_workflow.run(query=query)
result

Running step generate_sql_response
Step generate_sql_response produced event StopEvent


Response(response="Broadcom's investments are distributed as follows:  In Bond Funds, they have $1,445,430.40 and $15,911,558.77.  Their Equity Fund holdings total $2,931,233.29, $18,000,155.45, and $271,747.09.\n", source_nodes=[NodeWithScore(node=TextNode(id_='ad153ec8-768e-4c14-a7d5-c17a12bc68c9', embedding=None, metadata={'sql_query': "SELECT fund_type, fund_balance FROM portfolio WHERE company ~* '\\ybroadcom\\y'", 'result': [('Bond Fund', 1445430.4), ('Bond Fund', 15911558.77), ('Equity Fund', 2931233.29), ('Equity Fund', 18000155.45), ('Equity Fund', 271747.09)], 'col_keys': ['fund_type', 'fund_balance']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="[('Bond Fund', 1445430.4), ('Bond Fund', 15911558.77), ('Equity Fund', 2931233.29), ('Equity Fund', 18000155.45), ('Equity Fund', 271747.09)]", mimetype='text/plain

In [13]:
query = "what employees work on the mariott client account?"
result = await sql_workflow.run(query=query)
result

Running step generate_sql_response
Step generate_sql_response produced event StopEvent


Response(response="Here's a list of employees who work on the Marriott client account: Debra Kelly, Jacob Jennings, Alexandria Collins, Laura Brown, Michelle Mcbride, Michael Wilson, Julie Jimenez, Peter Marshall, Kelsey Charles, Catherine Hunter, Julie Harmon, Sergio Gonzalez, Tracy Willis, Lucas Cohen, Susan Johnson, Phillip Dudley, Carlos Marquez, Nicholas Rogers, Charles Wiggins, Michelle Trevino, and Jonathan Foster.\n", source_nodes=[NodeWithScore(node=TextNode(id_='11b83412-bd40-4ac7-9f29-da5041717980', embedding=None, metadata={'sql_query': "SELECT employee_first_name, employee_last_name FROM client_alignment WHERE company ~* '\\ymariott\\y'", 'result': [('Debra', 'Kelly'), ('Jacob', 'Jennings'), ('Alexandria', 'Collins'), ('Laura', 'Brown'), ('Michelle', 'Mcbride'), ('Michael', 'Wilson'), ('Julie', 'Jimenez'), ('Peter', 'Marshall'), ('Kelsey', 'Charles'), ('Catherine', 'Hunter'), ('Julie', 'Harmon'), ('Sergio', 'Gonzalez'), ('Tracy', 'Willis'), ('Lucas', 'Cohen'), ('Susan', 'J

In [14]:
query = "what is the client email for Mariott?"
result = await sql_workflow.run(query=query)
result

Running step generate_sql_response
Step generate_sql_response produced event StopEvent


Response(response='The email address for Marriott is debra.butler@mariott.com.\n', source_nodes=[NodeWithScore(node=TextNode(id_='43e90c83-4b59-46be-9308-3c677746ee70', embedding=None, metadata={'sql_query': "SELECT email FROM clients_contact WHERE company ~* '\\yMariott\\y'", 'result': [('debra.butler@mariott.com',)], 'col_keys': ['email']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="[('debra.butler@mariott.com',)]", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)], metadata={'43e90c83-4b59-46be-9308-3c677746ee70': {'sql_query': "SELECT email FROM clients_contact WHERE company ~* '\\yMariott\\y'", 'result': [('debra.butler@mariott.com',)], 'col_keys': ['email']}, 'sql_query': "SELECT email FROM clients_contact WHERE com

# Check Answers accuracy

In [15]:
from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

In [16]:
DB_NAME = 'crm'
engine = create_pg_engine(
   db_name=DB_NAME
)

In [17]:
pd.read_sql("SELECT * FROM Employees limit 10", con=engine)

,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,971a5efe-d9e7-4594-961f-7d624d035185,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,582-476-3744x504,2023-07-05,Enterprise Investment,San Fransisco
1,0ff018e4-9b73-4c37-8410-123d36e98785,Russell,Sherman,Product Specialist,russell.sherman@bankwell.com,(505)883-2389,2021-07-02,Customer Support,Miami
2,9651dec3-a491-4de9-a5c1-a2eb9a108fc7,Veronica,West,Product Specialist,veronica.west@bankwell.com,001-463-327-6381x802,2023-12-13,Enterprise Investment,Denver
3,70394e4b-26a9-4df5-9b1b-225bbeb2f815,Marie,Howard,Product Specialist,marie.howard@bankwell.com,286.752.5398x91335,2024-10-21,Sales,Miami
4,10ad2baa-195e-4fd2-ad4f-bae65c42ded4,Debra,Kelly,Relationship Manager,debra.kelly@bankwell.com,001-559-219-4289x5144,2021-02-23,Sales,New York City
5,864bfcce-a6ed-4a06-a122-ad7de2f42f00,Meghan,Wiggins,Relationship Manager,meghan.wiggins@bankwell.com,001-417-895-2987,2020-02-25,Enterprise Investment,Denver
6,10441ece-8259-4c30-84b4-910f6e741c40,Christopher,Flores,Finance Assistant,christopher.flores@bankwell.com,(346)591-2928x64207,2022-07-12,Small Business Investment,Dallas
7,d9b830be-a029-407c-8a1f-56638f5882e5,Brandon,Hernandez,Finance Assistant,brandon.hernandez@bankwell.com,682-353-5657x1897,2022-06-30,Sales,Denver
8,ab5b781e-fd6b-4a2d-a321-921fa57f1d45,Kevin,Rocha,Financial Advisor,kevin.rocha@bankwell.com,+1-545-831-6349,2025-01-12,Enterprise Investment,Washington DC
9,05f48adb-c9b4-4869-b250-02e73264bd43,Karen,Smith,Relationship Manager,karen.smith@bankwell.com,783.799.0885x420,2023-10-04,Small Business Investment,Miami
